# Estudio de precios de la energía eléctrica en España (PVPC)

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sb
import xlrd
import datetime


In [6]:
# Abrimos uno de los archivos excel de los precios diarios para ver cómo está organizado
book = xlrd.open_workbook("../Data/PVPC_DETALLE_DD_20240302.xls")

WARNING *** file size (62466) not 512 + multiple of sector size (512)


In [7]:
# Comprobación del libro y los datos que queremos extraer:
print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))
sh = book.sheet_by_index(0)
print("{0} {1} {2}".format(sh.name, sh.nrows, sh.ncols))
print("Cell E29 is {0}".format(sh.cell_value(rowx=28, colx=4)))
for rx in range(4,sh.nrows):
    for cx in range(6):
        print("{0}".format(sh.cell_value(rowx=rx, colx=cx)))

The number of worksheets is 2
Worksheet name(s): ['Tabla de Datos PCB', 'Tabla de Datos CYM']
Tabla de Datos PCB 29 34
Cell E29 is 42.9989463784
Día
Hora
Peaje
Periodo
Término energía PVPC
FEU = TEU + TCU
€/MWh consumo
Peajes y cargos TEU
€/MWh consumo
45353.0
1.0
2.0TD
3.0
45.6262381345
2.752
45353.0
2.0
2.0TD
3.0
45.545985781
2.752
45353.0
3.0
2.0TD
3.0
45.7915380494
2.752
45353.0
4.0
2.0TD
3.0
44.7289488055
2.752
45353.0
5.0
2.0TD
3.0
46.3218295617
2.752
45353.0
6.0
2.0TD
3.0
46.1003888055
2.752
45353.0
7.0
2.0TD
3.0
44.7307352933
2.752
45353.0
8.0
2.0TD
3.0
44.4820881345
2.752
45353.0
9.0
2.0TD
3.0
47.3576513047
2.752
45353.0
10.0
2.0TD
3.0
44.1382366091
2.752
45353.0
11.0
2.0TD
3.0
44.8773569626
2.752
45353.0
12.0
2.0TD
3.0
43.5729348284
2.752
45353.0
13.0
2.0TD
3.0
43.3362368284
2.752
45353.0
14.0
2.0TD
3.0
42.40935856
2.752
45353.0
15.0
2.0TD
3.0
42.6538066942
2.752
45353.0
16.0
2.0TD
3.0
42.9163553653
2.752
45353.0
17.0
2.0TD
3.0
45.959520524
2.752
45353.0
18.0
2.0TD
3.0
46.565

In [25]:
# Creamos un DataFrame en el que guardar los datos. Para ello utilizamos la fila que describe los datos posteriores, es decir la fila 5
#  y nos quedamos solo con los datos hasta la columna G ya que el resto no nos interesan.
columns = []
for cx in range(7):
        columns.append("{0}".format(sh.cell_value(rowx=4, colx=cx)))
columns



['Día',
 'Hora',
 'Peaje',
 'Periodo',
 'Término energía PVPC\nFEU = TEU + TCU\n€/MWh consumo',
 'Peajes y cargos TEU\n€/MWh consumo',
 'Precio producción\nTCU=CP x(1+PERD/100)\n€/MWh consumo']

In [57]:
# Creamos el dataframe que guardará todos los datos recogidos de los archivos diarios de los precios
df = pd.DataFrame(columns=['Día',
    'Hora',
    'Peaje',
    'Periodo',
    'Término energía PVPC\nFEU = TEU + TCU\n€/MWh consumo',
    'Peajes y cargos TEU\n€/MWh consumo',
    'Precio producción\nTCU=CP x(1+PERD/100)\n€/MWh consumo'])

df = df.astype({
    'Hora': 'float',
    'Peaje': 'object',
    'Periodo': 'object',
    'Término energía PVPC\nFEU = TEU + TCU\n€/MWh consumo': 'float',
    'Peajes y cargos TEU\n€/MWh consumo': 'float',
    'Precio producción\nTCU=CP x(1+PERD/100)\n€/MWh consumo': 'float'
})

df['Día'] = pd.to_datetime(df['Día'], errors='coerce')
#df['Hora'] = pd.to_datetime(df["Hora"], format="%H:%M:%S").dt.time
df

,Día,Hora,Peaje,Periodo,Término energía PVPC\nFEU = TEU + TCU\n€/MWh consumo,Peajes y cargos TEU\n€/MWh consumo,Precio producción\nTCU=CP x(1+PERD/100)\n€/MWh consumo


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 7 columns):
 #   Column                                                Non-Null Count  Dtype         
---  ------                                                --------------  -----         
 0   Día                                                   0 non-null      datetime64[ns]
 1   Hora                                                  0 non-null      float64       
 2   Peaje                                                 0 non-null      object        
 3   Periodo                                               0 non-null      object        
 4   Término energía PVPC
FEU = TEU + TCU
€/MWh consumo    0 non-null      float64       
 5   Peajes y cargos TEU
€/MWh consumo                     0 non-null      float64       
 6   Precio producción
TCU=CP x(1+PERD/100)
€/MWh consumo  0 non-null      float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 132.0+ bytes


In [58]:
# Guardamos este dataframe en un nuevo csv que va a recoger todos los precios de los diferentes excel diarios.DirEntry
df.to_csv("../PVPC_diarios.csv")

In [10]:
# Para extraer los datos de las filas que nos interesan del archivo y guardarlos en el dataframe donde almacenaremos todo el histórico
for rx in range(5,sh.nrows):
    row = []
    for cx in range(7):
        row.append("{0}".format(sh.cell_value(rowx=rx, colx=cx)))
    row_df = pd.DataFrame([row], columns=df.columns)
    # Convertir texto a número
    for col in row_df.columns[:2]:
        row_df[col] = pd.to_numeric(row_df[col])
    for col in row_df.columns[4:]:
        row_df[col] = pd.to_numeric(row_df[col])

    # Convertir texto a fecha
    row_df["Día"] = pd.to_datetime(row_df["Día"], origin="1899-12-30", unit="D", errors="coerce")

    # Convertir texto a hora
    row_df["Hora"] = row_df["Hora"].astype(int)
    row_df["Hora"] = row_df["Hora"].apply(lambda x: "00:00:00" if x == 24 else f"{x}:00:00")
    row_df["Hora"] = pd.to_datetime(row_df["Hora"], format="%H:%M:%S").dt.time

    # Concatena las filas a el dataframe completo
    df = pd.concat([df, row_df], ignore_index=True)

In [11]:
df

,Día,Hora,Peaje,Periodo,Término energía PVPC\nFEU = TEU + TCU\n€/MWh consumo,Peajes y cargos TEU\n€/MWh consumo,Precio producción\nTCU=CP x(1+PERD/100)\n€/MWh consumo
0,2024-03-02,01:00:00,2.0TD,3.0,45.626238,2.752,42.874238
1,2024-03-02,02:00:00,2.0TD,3.0,45.545986,2.752,42.793986
2,2024-03-02,03:00:00,2.0TD,3.0,45.791538,2.752,43.039538
3,2024-03-02,04:00:00,2.0TD,3.0,44.728949,2.752,41.976949
4,2024-03-02,05:00:00,2.0TD,3.0,46.321830,2.752,43.569830
5,2024-03-02,06:00:00,2.0TD,3.0,46.100389,2.752,43.348389
6,2024-03-02,07:00:00,2.0TD,3.0,44.730735,2.752,41.978735
7,2024-03-02,08:00:00,2.0TD,3.0,44.482088,2.752,41.730088
8,2024-03-02,09:00:00,2.0TD,3.0,47.357651,2.752,44.605651
9,2024-03-02,10:00:00,2.0TD,3.0,44.138237,2.752,41.386237


In [12]:
os.getcwd()

'c:\\Users\\anusk\\Documents\\My_projects\\Electric_energy\\Notebooks'

In [13]:
print(os.listdir())

['draftnotes.ipynb']


In [14]:
os.chdir("../")

In [15]:
print(os.listdir())

['Data', 'Notebooks', 'PVPC_diarios.csv']


In [17]:
os.chdir("Data/")

In [18]:
os.getcwd()

'c:\\Users\\anusk\\Documents\\My_projects\\Electric_energy\\Data'

In [59]:

for i in os.listdir():
    if i.endswith('.xls'):
        book = xlrd.open_workbook(f"../Data/{i}")
        sh = book.sheet_by_index(0)
        print(i)
    for rx in range(5,sh.nrows):
        row = []        
        for cx in range(7):
                row.append("{0}".format(sh.cell_value(rowx=rx, colx=cx)))
        row_df = pd.DataFrame([row], columns=df.columns)
            # Convertir texto a número
        for col in row_df.columns[:2]:
            row_df[col] = pd.to_numeric(row_df[col])
        for col in row_df.columns[4:]:
            row_df[col] = pd.to_numeric(row_df[col])

        # Convertir texto a fecha
        row_df["Día"] = pd.to_datetime(row_df["Día"], origin="1899-12-30", unit="D", errors="coerce").dt.date

        # Convertir texto a hora. Esto sería lo correcto, pero tenemos problema con el día en que se produce el cambio de hora, ya que hay 25 horas en vez de 25, por tanto para facilitar el análisis, 
        # lo dejamos en formato número por el momento
        # row_df["Hora"] = row_df["Hora"].astype(int)
        # row_df["Hora"] = row_df["Hora"].apply(lambda x: "00:00:00" if x == 24 else f"{x}:00:00")
        # row_df["Hora"] = pd.to_datetime(row_df["Hora"], format="%H:%M:%S").dt.time
        df = pd.concat([df, row_df], ignore_index=True)



WARNING *** file size (62466) not 512 + multiple of sector size (512)
PVPC_DETALLE_DD_20240302.xls
WARNING *** file size (62466) not 512 + multiple of sector size (512)
PVPC_DETALLE_DD_20240303.xls
WARNING *** file size (62466) not 512 + multiple of sector size (512)
PVPC_DETALLE_DD_20240304.xls
WARNING *** file size (62466) not 512 + multiple of sector size (512)
PVPC_DETALLE_DD_20240305.xls
WARNING *** file size (62466) not 512 + multiple of sector size (512)
PVPC_DETALLE_DD_20240306.xls
WARNING *** file size (62466) not 512 + multiple of sector size (512)
PVPC_DETALLE_DD_20240307.xls
WARNING *** file size (62466) not 512 + multiple of sector size (512)
PVPC_DETALLE_DD_20240308.xls
WARNING *** file size (62466) not 512 + multiple of sector size (512)
PVPC_DETALLE_DD_20240309.xls
WARNING *** file size (62466) not 512 + multiple of sector size (512)
PVPC_DETALLE_DD_20240310.xls
WARNING *** file size (62466) not 512 + multiple of sector size (512)
PVPC_DETALLE_DD_20240311.xls
WARNING **

In [60]:
df

,Día,Hora,Peaje,Periodo,Término energía PVPC\nFEU = TEU + TCU\n€/MWh consumo,Peajes y cargos TEU\n€/MWh consumo,Precio producción\nTCU=CP x(1+PERD/100)\n€/MWh consumo
0,2024-03-02,1.0,2.0TD,3.0,45.626238,2.752,42.874238
1,2024-03-02,2.0,2.0TD,3.0,45.545986,2.752,42.793986
2,2024-03-02,3.0,2.0TD,3.0,45.791538,2.752,43.039538
3,2024-03-02,4.0,2.0TD,3.0,44.728949,2.752,41.976949
4,2024-03-02,5.0,2.0TD,3.0,46.321830,2.752,43.569830
...,...,...,...,...,...,...,...
9043,2025-03-13,20.0,2.0TD,1.0,285.138123,92.539,192.599123
9044,2025-03-13,21.0,2.0TD,1.0,270.320654,92.539,177.781654
9045,2025-03-13,22.0,2.0TD,1.0,242.888634,92.539,150.349634
9046,2025-03-13,23.0,2.0TD,2.0,152.546754,28.201,124.345754


### WEBSCRAPPING PRUEBAS

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import re
import time
import pandas as pd
from datetime import datetime
from selenium.webdriver.common.keys import Keys

In [16]:
search_type = "datos"
search_start_date = datetime.strptime("1/1/2022", "%d/%m/%Y").strftime("%#d-%#m-%Y")
search_end_date = datetime.strptime("31/1/2022", "%d/%m/%Y").strftime("%#d-%#m-%Y")

In [17]:
cr_dr_path = "../Utils/chromedriver.exe"
service = Service(executable_path=cr_dr_path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(f"https://www.esios.ree.es/es/descargas?date_type={search_type}&start_date={search_start_date}&end_date={search_end_date}")

In [19]:
# Para aceptar solo las cookies necesarias
loadMore = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[4]/div/div[2]/button[4]')
loadMore.click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=134.0.6998.118)
Stacktrace:
	GetHandleVerifier [0x00007FF6E8A6FE65+26629]
	(No symbol) [0x00007FF6E89D6030]
	(No symbol) [0x00007FF6E886914C]
	(No symbol) [0x00007FF6E88C0D24]
	(No symbol) [0x00007FF6E88B28A4]
	(No symbol) [0x00007FF6E88E7AAA]
	(No symbol) [0x00007FF6E88B2156]
	(No symbol) [0x00007FF6E88E7CC0]
	(No symbol) [0x00007FF6E8910169]
	(No symbol) [0x00007FF6E88E7883]
	(No symbol) [0x00007FF6E88B0550]
	(No symbol) [0x00007FF6E88B1803]
	GetHandleVerifier [0x00007FF6E8DC72DD+3529853]
	GetHandleVerifier [0x00007FF6E8DDDA42+3621858]
	GetHandleVerifier [0x00007FF6E8DD24F3+3575443]
	GetHandleVerifier [0x00007FF6E8B3B79A+860474]
	(No symbol) [0x00007FF6E89E08AF]
	(No symbol) [0x00007FF6E89DCBE4]
	(No symbol) [0x00007FF6E89DCD86]
	(No symbol) [0x00007FF6E89CC2E9]
	BaseThreadInitThunk [0x00007FFD36A2E8D7+23]
	RtlUserThreadStart [0x00007FFD36C9BF6C+44]


In [ ]:
# Con esto buscamos los archivos relativos a la descarga de precios
buscador = driver.find_element(By.XPATH, '/html/body/main/div/div[2]/div/div[4]/div/div[1]/div[2]/div[1]/input')
buscador.send_keys('pvpc')
buscador.submit()

WebDriverException: Message: To submit an element, it must be nested inside a form element


In [ ]:
# Aquí se busca el elemento que iniciará la descarga y se le clicará
elemento_descarga = driver.find_element(By.XPATH, '/html/body/main/div/div[2]/div/div[4]/div/div[1]/div[4]/table/tbody/tr[3]/td[1]/a')
elemento_descarga.click()